## 基础设置
---
首先我们应该安装 duckdb 的 python 环境, 请自行上网搜索

In [56]:
import duckdb

随后根据我们的 ER 图构建我们的表, 构建表的 sql 文件在本地目录下

In [57]:
con = duckdb.connect(':memory:') # connect to memory 

sql_file_path = './table_construct.sql'
with open(sql_file_path, 'r') as sql_file:
    sql_script = sql_file.read()
    con.execute(sql_script)

## Table Overview
---
![er-model](./er-model.png)

简单测试一下, 首先我们先打印出数据库中所有表

In [58]:
# Query the table infomation of current database
result = con.sql("SELECT table_name FROM information_schema.tables WHERE table_schema = 'main'").fetchall()

# Show output
for table in result:
    print(table[0])

appointment
contains
employee
include
order
owner
pet
product
product_info
service
service_type


结果符合我们的预期

接下来我们任选一个表检查一下 schema

In [59]:
con.sql("DESCRIBE service")

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ id          │ INTEGER     │ NO      │ PRI     │ NULL    │ NULL    │
│ employee_id │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ type        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

Perfect! 由于没有填入数据, 因此表为 0 行, 但我们可以看到表的结构也是符合我们的 E-R 图的.


## 填入数据, 使用数据库

In [60]:
sql_file_path = './pad_data.sql'
with open(sql_file_path, 'r') as sql_file:
    sql_script = sql_file.read()
    con.execute(sql_script)

## Demo
---
### 示例 1: 格雷🦊斯表面是个黑帮教父, 实际是个宠物控, 让我们来看看格雷🦊斯在宠物店的订单记录

神奇的 Sql 代码如下
``` sql
SELECT "order".id, "order".time, "order".amount, "order".status
FROM "order"
JOIN owner ON "order".owner_id = owner.id
WHERE owner.name = '张三';
```

In [61]:
con.sql('''
    SELECT "order".id, "order".time, "order".amount, "order".status
    FROM "order"
    JOIN owner ON "order".owner_id = owner.id
    WHERE owner.name = '格雷🦊斯'
''')

┌───────┬─────────────────────┬────────┬─────────┐
│  id   │        time         │ amount │ status  │
│ int32 │      timestamp      │ float  │ varchar │
├───────┼─────────────────────┼────────┼─────────┤
│     1 │ 2023-01-20 14:00:00 │   50.0 │ 已完成  │
│     4 │ 2023-04-10 11:10:00 │   80.0 │ 已完成  │
└───────┴─────────────────────┴────────┴─────────┘

震惊😱 格雷🦊斯居然在宠物店里消费过这么多次
> “黑帮首领也有心肠。” - 阿尔·卡彭 (Al Capone)

### 示例2: 炸弹狂人💣吉格斯居然在宠物店接私活, 快来一探究竟

万能的 sql 代码如下
```sql
SELECT service_type.type, appointment.appoint_time
FROM service
JOIN employee ON service.employee_id = employee.id
JOIN service_type ON service.type = service_type.type
JOIN appointment ON service.id = appointment.id
WHERE employee.name = '吉格斯';
```

In [62]:
con.sql('''
    SELECT service_type.type, employee.position
    FROM service
    JOIN employee ON service.employee_id = employee.id
    JOIN service_type ON service.type = service_type.type
    WHERE employee.name = '吉格斯';
''')

┌─────────┬────────────┐
│  type   │  position  │
│ varchar │  varchar   │
├─────────┼────────────┤
│ 洗澡    │ 宠物护理师 │
└─────────┴────────────┘

想不到啊想不到, 炸弹人你居然去当了宠物护理师.

接下来是最后一个例子



### 示例三 伊泽瑞尔是宠物店的常客, 但是经常忘记自己买过什么东西, 请你来帮帮他

Powerful 的 sql 代码如下:
```sql
SELECT product_info.name, "order".amount, product.stock_quantity
FROM "order"
JOIN owner ON "order".owner_id = owner.id
JOIN contains ON "order".id = contains.order_id
JOIN product ON contains.product_id = product.id
JOIN product_info ON product.name = product_info.name
WHERE owner.name = '伊泽瑞尔';

```

In [63]:
con.sql('''
    SELECT product_info.name, "order".amount, product.stock_quantity
    FROM "order"
    JOIN owner ON "order".owner_id = owner.id
    JOIN contains ON "order".id = contains.order_id
    JOIN product ON contains.product_id = product.id
    JOIN product_info ON product.name = product_info.name
    WHERE owner.name = 'EZ瑞尔';
''')

┌──────────┬────────┬────────────────┐
│   name   │ amount │ stock_quantity │
│ varchar  │ float  │     int32      │
├──────────┼────────┼────────────────┤
│ 宠物玩具 │  100.0 │             50 │
└──────────┴────────┴────────────────┘

哦吼, EZ 瑞尔买过宠物玩具, 一共花了 100 大米 (1/4.5)个多兰剑, 真奢侈!



## 结束
最后, 不要忘记关闭我们的数据库 🥰, 展示完毕! see you again

In [64]:
con.close()